In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing_extensions import List
from scipy.signal import find_peaks

@dataclass
class PeakData:

    index: str
    x: np.ndarray
    y: np.ndarray

    @property
    def time_diff(self) -> List[int]:
        deltas = []
        for i in range(len(self.x)-1):
            deltas.append(self.x[i+1] - self.x[i])
        return np.array(deltas, dtype=float)

@dataclass
class Peaks:

    good: List[PeakData]
    less: List[PeakData]
    more: List[PeakData]

@dataclass
class MuonDecayEvents:

    df: pd.DataFrame

    @property
    def scale(self) -> float:
        """mu-sec/div"""
        return 1E-6

    @property
    def number_divisions(self) -> int:
        return 10

    @property
    def number_adc_channels(self) -> int:
        return len(self.df.columns)

    @property
    def time_data(self) -> np.ndarray:
        data = np.array([i for i in range(self.number_adc_channels)])
        return self.convert_time_to_units(time_data=data)

    def convert_time_to_units(self, time_data: np.ndarray) -> np.ndarray:
        k = self.number_divisions / self.number_adc_channels
        return k * time_data

    def find_peaks(self, height: float = -10., distance: int = 10, invert_data: bool = False) -> Peaks:
        k = -1 if invert_data else 1
        good = []
        less = []
        more = []
        for index in self.df.index:
            x_peaks, y_peaks = find_peaks(x=k*self.df.loc[index], height=height, distance=distance)
            peaks = PeakData(
                index = index,
                x     = self.convert_time_to_units(x_peaks), 
                y     = k*y_peaks['peak_heights'],
            )
            if len(x_peaks) == 2:
                good.append(peaks)
            elif len(x_peaks) < 2:
                less.append(peaks)
            elif len(x_peaks) > 2:
                more.append(peaks)
        return Peaks(good=good, less=less, more=more)

    def plot_event(self, peaks_data: PeakData) -> None:
        plt.xlabel(xlabel=r"Tempo ($\mu$s)")
        plt.ylabel(ylabel="Tensão (chADC)")
        plt.plot(
            self.time_data,
            self.df.loc[peaks_data.index],
            color="#696969",
        )
        plt.scatter(
            x      = peaks_data.x, 
            y      = peaks_data.y,
            c      = "red",
            marker = "x",
        )
        plt.savefig("ex_waveform.png", dpi=500)

In [ ]:
df = pd.read_csv(filepath_or_buffer="5555_eventos_T.csv", header=0, index_col=0, sep=",")

df

In [ ]:
decay = MuonDecayEvents(df=df)

In [ ]:
peaks = decay.find_peaks(height=-10., distance=10, invert_data=True)

In [ ]:
peaks.good[100].time_diff

In [ ]:
decay.plot_event(peaks_data=peaks.good[400])

In [ ]:
time_diff = np.array([
    peaks.good[i].time_diff[0] \
    for i in range(len(peaks.good))
])

time_diff

In [ ]:
df_time_diff = pd.DataFrame(
    data    = time_diff, 
    columns = ["time_micro_sec"],
    index   = df.index,
)

df_time_diff.to_csv(path_or_buf="time_diff.csv", sep=",", index=False)

In [ ]:
from scipy.optimize import curve_fit

def exp_function(x: float, a: float, tau: float, c: float) -> float:
    return a * np.exp(-x/tau) + c

In [ ]:
hist, bin_edges = np.histogram(a=time_diff, bins="fd")
bin_centers = np.array([
    (bin_edges[i] + bin_edges[i+1])/2 for i in range(len(bin_edges)-1) 
])
popt, pcov = curve_fit(f=exp_function, xdata=bin_centers, ydata=hist)

In [ ]:
plt.hist(x=time_diff, density=True, bins="fd", color="black", histtype='step')
plt.xlabel(r"Diferença de tempo ($\mu$s)")
plt.ylabel('Probability')
plt.show()